DATASET PREIVIEW

In [21]:
from datasets import load_from_disk
asdiv = load_from_disk("datasets/data/hf/asdiv_local")
print("Asdiv")
print(asdiv)
for i in range(5):
    print(asdiv[i])

Asdiv
Dataset({
    features: ['body', 'question', 'solution_type', 'answer', 'formula'],
    num_rows: 2305
})
{'body': 'Seven red apples and two green apples are in the basket.', 'question': 'How many apples are in the basket?', 'solution_type': 'Addition', 'answer': '9 (apples)', 'formula': '7+2=9'}
{'body': 'Ellen has six more balls than Marin. Marin has nine balls.', 'question': 'How many balls does Ellen have?', 'solution_type': 'Addition', 'answer': '15 (balls)', 'formula': '6+9=15'}
{'body': 'Janet has nine oranges and Sharon has seven oranges.', 'question': 'How many oranges do Janet and Sharon have together? ', 'solution_type': 'Addition', 'answer': '16 (oranges)', 'formula': '9+7=16'}
{'body': 'Allan brought two balloons and Jake brought four balloons to the park.', 'question': 'How many balloons did Allan and Jake have in the park?', 'solution_type': 'Addition', 'answer': '6 (balloons)', 'formula': '2+4=6'}
{'body': 'Adam has five more apples than Jackie. Jackie has nine ap

In [22]:
import json
# train.json
with open("datasets/data/mathqa/train.json", "r") as f:
    mathqa_train = json.load(f)
    
print("mathqa")
print("Number of samples:", len(mathqa_train))
print("First sample keys:", mathqa_train[0].keys())
for i in range(5):
    print("First sample:", mathqa_train[i])


mathqa
Number of samples: 29837
First sample keys: dict_keys(['Problem', 'Rationale', 'options', 'correct', 'annotated_formula', 'linear_formula', 'category'])
First sample: {'Problem': "the banker ' s gain of a certain sum due 3 years hence at 10 % per annum is rs . 36 . what is the present worth ?", 'Rationale': '"explanation : t = 3 years r = 10 % td = ( bg × 100 ) / tr = ( 36 × 100 ) / ( 3 × 10 ) = 12 × 10 = rs . 120 td = ( pw × tr ) / 100 ⇒ 120 = ( pw × 3 × 10 ) / 100 ⇒ 1200 = pw × 3 pw = 1200 / 3 = rs . 400 answer : option a"', 'options': 'a ) rs . 400 , b ) rs . 300 , c ) rs . 500 , d ) rs . 350 , e ) none of these', 'correct': 'a', 'annotated_formula': 'divide(multiply(const_100, divide(multiply(36, const_100), multiply(3, 10))), multiply(3, 10))', 'linear_formula': 'multiply(n2,const_100)|multiply(n0,n1)|divide(#0,#1)|multiply(#2,const_100)|divide(#3,#1)|', 'category': 'gain'}
First sample: {'Problem': 'average age of students of an adult school is 40 years . 120 new students 

In [23]:
mawps = load_from_disk("datasets/data/mawps_dataset/train")
print("mwaps")
print(mawps)
for i in range(5):
    print(mawps[i])   # preview first row

mwaps
Dataset({
    features: ['id', 'question', 'chain', 'result', 'result_float', 'equation', 'expression'],
    num_rows: 1089
})
{'id': 'mawps__8kCVLDe7ByufbvB8', 'question': 'Mark had 2 Doll. Roland proffered him some more. Now Mark has 161 Doll. How many did Roland proffer him?', 'chain': '<gadget id="calculator">161 - 2</gadget>\n<output>159</output>\n\n<result>159</result>', 'result': '159', 'result_float': 159.0, 'equation': 'x = 161 - 2', 'expression': '161-2'}
{'id': 'mawps__7bhf7hPk6pDUOioA', 'question': 'Gloria had some raspberry. Margaret gave him 7 more. Now Gloria has 33 raspberry. How many raspberry did Gloria have in the first?', 'chain': '<gadget id="calculator">33 - 7</gadget>\n<output>26</output>\n\n<result>26</result>', 'result': '26', 'result_float': 26.0, 'equation': 'x = 33 - 7', 'expression': '33-7'}
{'id': 'mawps__8H7IbSn9KDIuz1S3', 'question': 'Tina had 7 raspberry . He hash each raspberry into 10 slices . How many raspberry slices did Tina make?', 'chain': 

In [25]:
# Notebook cell 1: imports & paths
import json, re, os
from collections import Counter, defaultdict
import pandas as pd
from pathlib import Path

# --- Adjust if needed ---
ROOT = Path("datasets/data/mathqa")
TRAIN_PATH = ROOT / "train.json"
DEV_PATH   = ROOT / "dev.json"
TEST_PATH  = ROOT / "test.json"
CONST_PATH = ROOT / "constant_list.txt"
OP_PATH    = ROOT / "operation_list.txt"

OUTDIR = ROOT / "processed"
OUTDIR.mkdir(parents=True, exist_ok=True)

print("Paths:")
print(" train:", TRAIN_PATH.exists(), TRAIN_PATH)
print(" dev:  ", DEV_PATH.exists(), DEV_PATH)
print(" test: ", TEST_PATH.exists(), TEST_PATH)
print(" const:", CONST_PATH.exists(), CONST_PATH)
print(" ops:  ", OP_PATH.exists(), OP_PATH)


Paths:
 train: True datasets\data\mathqa\train.json
 dev:   True datasets\data\mathqa\dev.json
 test:  True datasets\data\mathqa\test.json
 const: True datasets\data\mathqa\constant_list.txt
 ops:   True datasets\data\mathqa\operation_list.txt


In [26]:
# Notebook cell 2: load constants & operations
def load_lines(path):
    if not path.exists():
        return []
    with open(path, "r", encoding="utf-8") as f:
        lines = [ln.strip() for ln in f if ln.strip()]
    return lines

constants = load_lines(CONST_PATH)
operations = load_lines(OP_PATH)

# normalize constants and ops
constants_set = set([c.strip() for c in constants])
ops_set = set([o.strip() for o in operations])

print("Loaded constants:", len(constants_set))
print("First constants:", list(constants_set)[:10])
print("Loaded operations:", len(ops_set))
print("First operations:", list(ops_set)[:20])


Loaded constants: 23
First constants: ['CONST_2', 'CONST_1.6', 'CONST_10', 'CONST_180', 'CONST_0.3937', 'CONST_3.6', 'CONST_6', 'CONST_0.25', 'CONST_DEG_TO_RAD', 'CONST_3']
Loaded operations: 77
First operations: ['subtract', 'max', 'semi_circle_perimiter', 'square_edge_by_perimeter', 'permutation', 'min', 'reminder', 'surface_cylinder', 'trapezium_area', 'triangle_area', 'gain_percent', 'sum_consecutive_number', 'stream_speed', 'volume_rectangular_prism', 'count_interval', 'triangle_area_three_edges', 'cube_edge_by_volume', 'add', 'sqrt', 'p_after_gain']


In [27]:
# Notebook cell 3: helper canonicalization functions
# We'll map:
#  - n#   -> ARG#
#  - const_name  -> CONST_NAME (upper)
#  - #k   -> TEMPk
token_map_patterns = [
    (re.compile(r"\bn(\d+)\b"), lambda m: f"ARG{m.group(1)}"),
    (re.compile(r"\bconst_([A-Za-z0-9_.]+)\b"), lambda m: f"CONST_{m.group(1).upper().replace('.','_')}"),
    (re.compile(r"\#(\d+)"), lambda m: f"TEMP{m.group(1)}"),
]

def canonicalize_linear_formula(linear, validate_ops=True):
    """
    Convert linear_formula tokens into canonical placeholders.
    Optionally validate presence of known operations (not required, but useful).
    """
    if not linear:
        return ""
    s = linear.strip().strip('|')
    # apply token replacements
    for pat, fn in token_map_patterns:
        s = pat.sub(fn, s)
    # optionally validate operations: linear is pipe-separated tokens like "add(ARG0,ARG1)|multiply(ARG1,ARG2)"
    if validate_ops:
        parts = s.split("|")
        cleaned_parts = []
        for p in parts:
            # extract op name: beginning until '('
            op_match = re.match(r"\s*([a-zA-Z0-9_]+)\s*\(", p)
            if op_match:
                op = op_match.group(1)
                # allow a few variants: e.g. "divide" etc.
                if op not in ops_set:
                    # if unknown, keep but mark with UNKOP_ prefix
                    p = p.replace(op, "UNKOP_" + op)
            cleaned_parts.append(p)
        s = "|".join(cleaned_parts)
    s = re.sub(r"\s*\|\s*", "|", s)
    s = re.sub(r"\s+", " ", s)
    return s

def canonicalize_annotated_formula(annot):
    """
    Convert annotated_formula where functions are nested into canonical tokens similarly.
    """
    if not annot:
        return ""
    s = annot
    for pat, fn in token_map_patterns:
        s = pat.sub(fn, s)
    s = re.sub(r"\s+", " ", s)
    return s


In [28]:
# Notebook cell 4: load JSON splits
def load_json(path):
    if not path.exists():
        return []
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

train = load_json(TRAIN_PATH)
dev   = load_json(DEV_PATH)
test  = load_json(TEST_PATH)
print("counts train/dev/test:", len(train), len(dev), len(test))


counts train/dev/test: 29837 4475 2985


In [29]:
# Notebook cell 5: process splits & build template bank
def clean_text(s):
    if s is None:
        return ""
    s = str(s).replace("\n", " ").replace("\t", " ").strip()
    s = re.sub(r"\s+", " ", s)
    return s

all_data = []
template_counter = Counter()
template_examples = defaultdict(list)

for split_name, split in [("train", train), ("dev", dev), ("test", test)]:
    for i, item in enumerate(split):
        # standard fields (some variants)
        problem = clean_text(item.get("Problem") or item.get("problem") or "")
        rationale = clean_text(item.get("Rationale") or item.get("rationale") or "")
        linear = item.get("linear_formula")
        annotated = item.get("annotated_formula")
        # canonicalize
        linear_canon = canonicalize_linear_formula(linear)
        annot_canon = canonicalize_annotated_formula(annotated)
        # prefer linear if available; otherwise annotated
        template_key = linear_canon if linear_canon else (annot_canon if annot_canon else "")
        eid = item.get("id", f"mathqa_{split_name}_{i}")
        record = {
            "id": eid,
            "split": split_name,
            "problem": problem,
            "rationale": rationale,
            "linear_formula": linear_canon,
            "annotated_formula": annot_canon,
            "raw_linear": linear,
            "raw_annotated": annotated,
            "original": item
        }
        all_data.append(record)
        if template_key:
            template_counter[template_key] += 1
            # store small example metadata for the template
            template_examples[template_key].append({
                "id": eid,
                "split": split_name,
                "problem_snippet": problem[:300]
            })

print("Total processed examples:", len(all_data))
print("Total unique templates found:", len(template_counter))


Total processed examples: 37297
Total unique templates found: 8762


In [30]:
# Notebook cell 6: create template bank dataframe & save top templates
TOP_N = 500  # adjust how many top templates to keep examples for

template_items = [{"template": t, "count": c, "examples": template_examples[t][:5]} 
                  for t, c in template_counter.most_common()]

template_df = pd.DataFrame([{"template": ti["template"], "count": ti["count"]} for ti in template_items])

# Save cleaned dataset (jsonl) and template summaries
cleaned_path = OUTDIR / "mathqa_cleaned_all_splits.jsonl"
with open(cleaned_path, "w", encoding="utf-8") as fout:
    for rec in all_data:
        # drop "original" if too big; we'll keep main fields
        out_rec = {
            "id": rec["id"],
            "split": rec["split"],
            "problem": rec["problem"],
            "rationale": rec["rationale"],
            "linear_formula": rec["linear_formula"],
            "annotated_formula": rec["annotated_formula"]
        }
        fout.write(json.dumps(out_rec, ensure_ascii=False) + "\n")

template_csv = OUTDIR / "template_bank_summary.csv"
template_df.to_csv(template_csv, index=False)

top_examples_path = OUTDIR / "top_templates_examples.json"
top_templates_json = {t: template_examples[t][:10] for t, _ in template_counter.most_common(TOP_N)}
with open(top_examples_path, "w", encoding="utf-8") as fout:
    json.dump(top_templates_json, fout, ensure_ascii=False, indent=2)

print("Saved cleaned dataset:", cleaned_path)
print("Saved template summary CSV:", template_csv)
print("Saved a JSON of top templates/examples:", top_examples_path)


Saved cleaned dataset: datasets\data\mathqa\processed\mathqa_cleaned_all_splits.jsonl
Saved template summary CSV: datasets\data\mathqa\processed\template_bank_summary.csv
Saved a JSON of top templates/examples: datasets\data\mathqa\processed\top_templates_examples.json


In [31]:
# Notebook cell 7: quick inspection - show top 20 templates and some stats
print("Top 20 templates (template => count):")
for t, c in template_counter.most_common(20):
    print(f"{c:5d}  {t}")

# show distribution of template lengths (simple)
lens = [len(t.split("|")) if t else 0 for t in template_counter.keys()]
s = pd.Series(lens)
print("\nTemplate length distribution (number of pipe-separated ops):")
print(s.describe())


Top 20 templates (template => count):
  241  divide(ARG0,ARG1)
  239  divide(ARG0,ARG1)|multiply(TEMP0,CONST_100)
  175  multiply(ARG0,ARG1)|divide(TEMP0,ARG2)
  175  multiply(ARG0,ARG2)
  165  add(ARG0,ARG1)|divide(TEMP0,CONST_2)
  149  divide(ARG1,ARG0)
  138  multiply(ARG0,ARG1)
  116  add(ARG0,CONST_1)
  110  multiply(ARG1,CONST_0_2778)|divide(ARG0,TEMP0)
  106  multiply(ARG0,ARG1)|add(ARG2,TEMP0)
  106  multiply(ARG0,CONST_1000)|divide(TEMP0,CONST_3600)|multiply(ARG1,TEMP1)
   93  subtract(ARG1,ARG0)|divide(TEMP0,ARG0)|multiply(TEMP1,CONST_100)
   86  add(ARG0,ARG1)
   82  negate(ARG3)|subtract(ARG1,ARG2)|subtract(ARG0,ARG1)|divide(TEMP1,TEMP2)|multiply(TEMP3,TEMP1)|subtract(TEMP0,TEMP4)
   81  add(ARG0,ARG2)|multiply(ARG1,CONST_0_2778)|divide(TEMP0,TEMP1)
   77  add(ARG0,ARG1)|add(ARG2,ARG3)|multiply(TEMP1,CONST_0_2778)|divide(TEMP0,TEMP2)
   73  subtract(ARG1,ARG2)|multiply(TEMP0,CONST_0_2778)|divide(ARG0,TEMP1)
   73  add(ARG1,CONST_4)|factorial(ARG1)|subtract(CONST_4,CONST_1)|

In [32]:
# Notebook cell 8: simple slot extractor (optional)
# This will show how many unique ARG# placeholders exist in top templates and their counts.
def extract_args(template):
    return sorted(set(re.findall(r"ARG\d+", template)))

top_k = 50
print("\nSample top templates with extracted ARG placeholders:")
for t, c in template_counter.most_common(top_k)[:20]:
    print(f"{c:5d}  args={extract_args(t)}  -> {t}")



Sample top templates with extracted ARG placeholders:
  241  args=['ARG0', 'ARG1']  -> divide(ARG0,ARG1)
  239  args=['ARG0', 'ARG1']  -> divide(ARG0,ARG1)|multiply(TEMP0,CONST_100)
  175  args=['ARG0', 'ARG1', 'ARG2']  -> multiply(ARG0,ARG1)|divide(TEMP0,ARG2)
  175  args=['ARG0', 'ARG2']  -> multiply(ARG0,ARG2)
  165  args=['ARG0', 'ARG1']  -> add(ARG0,ARG1)|divide(TEMP0,CONST_2)
  149  args=['ARG0', 'ARG1']  -> divide(ARG1,ARG0)
  138  args=['ARG0', 'ARG1']  -> multiply(ARG0,ARG1)
  116  args=['ARG0']  -> add(ARG0,CONST_1)
  110  args=['ARG0', 'ARG1']  -> multiply(ARG1,CONST_0_2778)|divide(ARG0,TEMP0)
  106  args=['ARG0', 'ARG1', 'ARG2']  -> multiply(ARG0,ARG1)|add(ARG2,TEMP0)
  106  args=['ARG0', 'ARG1']  -> multiply(ARG0,CONST_1000)|divide(TEMP0,CONST_3600)|multiply(ARG1,TEMP1)
   93  args=['ARG0', 'ARG1']  -> subtract(ARG1,ARG0)|divide(TEMP0,ARG0)|multiply(TEMP1,CONST_100)
   86  args=['ARG0', 'ARG1']  -> add(ARG0,ARG1)
   82  args=['ARG0', 'ARG1', 'ARG2', 'ARG3']  -> negate(ARG

In [46]:
# Cell 1: imports & paths
from pathlib import Path
import json, re, hashlib
from collections import defaultdict, Counter
import math
import pandas as pd

ROOT = Path("datasets/data/mathqa")
INPATH = ROOT / "processed" / "mathqa_cleaned_all_splits.jsonl"
OUTDIR = Path("FinalData")
OUTDIR.mkdir(parents=True, exist_ok=True)

print("Input:", INPATH.exists(), INPATH)
print("Outdir:", OUTDIR)


Input: True datasets\data\mathqa\processed\mathqa_cleaned_all_splits.jsonl
Outdir: FinalData


In [40]:
# Cell 2: small parser for annotated_formula -> AST
# annotated_formula example: "divide(90, subtract(const_1, divide(2, 3)))"
# Parser returns nested dicts: {"op":"divide","args":[...]} or leaf strings like "ARG0" or "CONST_100" or "90"

_tok_re = re.compile(r"\s*([A-Za-z0-9_.#]+)\s*(\(|,|\))?")

def parse_annotated_formula(s):
    """
    Recursive simple parser for function-call style annotated_formula.
    Returns AST nodes:
      - dict: {"op": op_name, "args": [arg1,...]}
      - leaf str: token like ARG0, CONST_100 or numeric literal
    """
    if not s or not isinstance(s, str):
        return None
    # remove surrounding whitespace
    s = s.strip()
    # inner recursive parser using index
    idx = 0
    n = len(s)

    def parse_expr(i):
        # parse token (op or literal)
        # token can be letters/numbers/._# then optionally '(' args ')'
        m = _tok_re.match(s, i)
        if not m:
            raise ValueError(f"Parse error at pos {i} in {s!r}")
        token = m.group(1)
        sep = m.group(2)
        j = m.end()
        if sep == "(":
            # parse args until matching ')'
            args = []
            # handle empty arg lists (rare)
            while True:
                # peek for closing parenthesis
                # skip whitespace
                while j < n and s[j].isspace():
                    j += 1
                if j < n and s[j] == ")":
                    j += 1
                    break
                # parse next arg
                arg_node, j = parse_expr(j)
                args.append(arg_node)
                # skip spaces
                while j < n and s[j].isspace():
                    j += 1
                # next char should be ',' or ')'
                if j < n and s[j] == ",":
                    j += 1
                    continue
                if j < n and s[j] == ")":
                    j += 1
                    break
                # otherwise continue (watch for end)
            return ({"op": token, "args": args}, j)
        else:
            # leaf token (could be numeric or const or ARG# or #temp)
            return (token, j)

    node, pos = parse_expr(0)
    # ensure consumed
    # pos may be less than n if trailing pipes exist (shouldn't for annotated)
    return node


In [41]:
# Cell 3: AST -> canonical template & AST serializer
def ast_to_template(node):
    """
    Convert AST node into canonical template string where ARG\d+ -> {ARG#}, const_* -> CONST_*, temps preserved.
    Returns serialized string (node-based) used as template key.
    """
    if node is None:
        return ""
    if isinstance(node, str):
        # map tokens
        tok = node.strip()
        # standardize ARGn -> {ARGn}, const_x -> CONST_X
        m = re.match(r"n(\d+)$", tok)  # sometimes n0 style (rare in annotated)
        if m:
            return "{" + "ARG" + m.group(1) + "}"
        m2 = re.match(r"ARG(\d+)$", tok)
        if m2:
            return "{" + "ARG" + m2.group(1) + "}"
        m3 = re.match(r"const_([A-Za-z0-9_.]+)$", tok, flags=re.I)
        if m3:
            return "CONST_" + m3.group(1).upper().replace(".", "_")
        # temp like #0 or TEMP0
        m4 = re.match(r"#(\d+)$", tok)
        if m4:
            return "TEMP" + m4.group(1)
        # else numeric literal or other token -> keep as-is
        return tok
    # dict node
    op = node.get("op")
    args = node.get("args", [])
    arg_strs = [ast_to_template(a) for a in args]
    return f"{op}(" + ",".join(arg_strs) + ")"


In [42]:
# Cell 4: AST -> infix conversion (best-effort)
_infix_map = {
    "add": ("{a} + {b}", 2),
    "subtract": ("{a} - {b}", 2),
    "multiply": ("{a} * {b}", 2),
    "divide": ("{a} / {b}", 2),
    "power": ("({a}) ** ({b})", 2),
    "negate": ("-({a})", 1),
    "sqrt": ("sqrt({a})", 1),
    "log": ("log({a})", 1),
    "max": ("max({a},{b})", 2),
    "min": ("min({a},{b})", 2),
    # if more operations required, extend here
}

def ast_to_infix(node):
    if node is None:
        return ""
    if isinstance(node, str):
        return node
    op = node.get("op")
    args = node.get("args", [])
    infix_args = [ast_to_infix(a) for a in args]
    if op in _infix_map:
        pattern, arity = _infix_map[op]
        if arity == 1 and len(infix_args) >= 1:
            return pattern.format(a=infix_args[0])
        if arity == 2 and len(infix_args) >= 2:
            return pattern.format(a=infix_args[0], b=infix_args[1])
    # default: function style
    return f"{op}(" + ",".join(infix_args) + ")"


In [43]:
# Cell 5: utilities to extract numeric tokens from problem text
_num_re = re.compile(r'(?:(?:\d+(?:\.\d+)?)|(?:\d+\s*/\s*\d+)|(?:\d+\.\d+))')

def extract_numbers_from_text(text):
    text = text or ""
    # return list of strings and parsed floats where possible
    nums = []
    for m in _num_re.finditer(text):
        raw = m.group(0)
        raw = raw.replace(" ", "")
        # handle fraction
        if "/" in raw:
            try:
                a,b = raw.split("/")
                val = float(a)/float(b)
            except Exception:
                val = None
        else:
            try:
                val = float(raw)
            except:
                val = None
        nums.append({"text": raw, "value": val, "span": m.span()})
    return nums


In [44]:
# Cell 6: main loop - read file, parse AST, build template bank & semantic specs
template_index = {}   # template_str -> template_id (hash)
template_data = {}    # template_id -> info
template_examples = defaultdict(list)
semantic_specs = []   # list of per-example semantic JSON

with open(INPATH, "r", encoding="utf-8") as fin:
    for line in fin:
        item = json.loads(line)
        eid = item.get("id")
        problem = item.get("problem", "")
        linear = item.get("linear_formula", "") or ""
        annotated = item.get("annotated_formula", "") or ""
        # prefer annotated_formula for AST parsing (linear is flattened)
        source_formula = annotated if annotated else linear
        try:
            ast = parse_annotated_formula(source_formula) if source_formula else None
        except Exception as e:
            ast = None
        template_str = ast_to_template(ast) if ast else ("LINEAR:" + linear if linear else "")
        # template id: hash shorter
        tid = hashlib.sha1(template_str.encode("utf-8")).hexdigest()[:12]
        if tid not in template_data:
            # initialize
            template_data[tid] = {
                "template_id": tid,
                "template_str": template_str,
                "ast_template": ast,   # best-effort; serializable?
                "count": 0,
                "ops": [],
                "constants": set(),
                "examples": []
            }
        # increment
        template_data[tid]["count"] += 1
        template_data[tid]["examples"].append(eid)
        # collect ops and constants: traverse AST
        ops = set()
        consts = set()
        argslots = set()
        def walk(n):
            if n is None:
                return
            if isinstance(n, str):
                tok = n
                # identify ARG, CONST, TEMP
                if re.match(r"ARG\d+", tok):
                    argslots.add(tok)
                if re.match(r"CONST_[A-Z0-9_\.]+", tok) or tok.lower().startswith("const_"):
                    consts.add(tok)
                return
            # dict
            opn = n.get("op")
            if opn:
                ops.add(opn)
            for a in n.get("args", []):
                walk(a)
        walk(ast)
        template_data[tid]["ops"] = sorted(list(ops))
        template_data[tid]["constants"] = sorted(list(consts))
        # prepare semantic spec for this example
        infix = ast_to_infix(ast) if ast else ""
        numbers_in_text = extract_numbers_from_text(problem)
        # simple mapping: ARG0->first number, ARG1->second...
        arg_to_number = {}
        # find all ARG tokens in AST and sort by arg index
        arg_tokens = sorted(set(re.findall(r"ARG(\d+)", template_str)), key=lambda x: int(x))
        for idx, arg_idx in enumerate(arg_tokens):
            key = "ARG" + arg_idx
            if idx < len(numbers_in_text):
                arg_to_number[key] = numbers_in_text[idx]["text"]
            else:
                arg_to_number[key] = None
        spec = {
            "id": eid,
            "template_id": tid,
            "template_str": template_str,
            "infix": infix,
            "ops": sorted(list(ops)),
            "constants": sorted(list(consts)),
            "arg_slots": arg_tokens,
            "arg_to_number_heuristic": arg_to_number,
            "problem_snippet": problem[:300]
        }
        semantic_specs.append(spec)

# finalize template list
template_list = list(template_data.values())
# convert sets to lists for JSON serialization
for t in template_list:
    if isinstance(t.get("constants"), set):
        t["constants"] = sorted(list(t["constants"]))


In [47]:
# Cell 7: save outputs (template_bank.json, template_bank_summary.csv, semantic_specs.jsonl)
tpl_json_path = OUTDIR / "template_bank.json"
tpl_csv_path  = OUTDIR / "template_bank_summary.csv"
specs_path    = OUTDIR / "semantic_specs.jsonl"

# Write template_bank.json (full)
with open(tpl_json_path, "w", encoding="utf-8") as f:
    json.dump(template_list, f, ensure_ascii=False, indent=2)

# Summary CSV
df_rows = []
for t in template_list:
    df_rows.append({
        "template_id": t["template_id"],
        "template_str": t["template_str"],
        "count": t["count"],
        "ops": ";".join(t.get("ops", [])),
        "num_examples": len(t.get("examples", [])),
        "constants": ";".join(t.get("constants", [])),
        "sample_example": t.get("examples", [None])[0]
    })
pd.DataFrame(df_rows).sort_values("count", ascending=False).to_csv(tpl_csv_path, index=False)

# semantic specs
with open(specs_path, "w", encoding="utf-8") as f:
    for s in semantic_specs:
        f.write(json.dumps(s, ensure_ascii=False) + "\n")

print("Wrote:", tpl_json_path)
print("Wrote:", tpl_csv_path)
print("Wrote:", specs_path)
print("Template count:", len(template_list))


Wrote: FinalData\template_bank.json
Wrote: FinalData\template_bank_summary.csv
Wrote: FinalData\semantic_specs.jsonl
Template count: 8762


In [48]:
# Cell 8: quick inspection - top templates
df = pd.read_csv(OUTDIR / "template_bank_summary.csv")
print("Top 20 templates by count:")
display(df.sort_values("count", ascending=False).head(20))


Top 20 templates by count:


,template_id,template_str,count,ops,num_examples,constants,sample_example
0,c5df09e2ebc6,"LINEAR:divide(ARG0,ARG1)",241,NaN,241,NaN,mathqa_train_4
1,d3e7fcc76529,"LINEAR:divide(ARG0,ARG1)|multiply(TEMP0,CONST_...",239,NaN,239,NaN,mathqa_train_3
2,6b1e389c2c80,"LINEAR:multiply(ARG0,ARG2)",175,NaN,175,NaN,mathqa_train_282
3,a0abfe4e3a5e,"LINEAR:multiply(ARG0,ARG1)|divide(TEMP0,ARG2)",175,NaN,175,NaN,mathqa_train_220
4,ba792035ceaa,"LINEAR:add(ARG0,ARG1)|divide(TEMP0,CONST_2)",165,NaN,165,NaN,mathqa_train_23
5,1379ef00d64f,"LINEAR:divide(ARG1,ARG0)",149,NaN,149,NaN,mathqa_train_52
6,39075bf05f53,"LINEAR:multiply(ARG0,ARG1)",138,NaN,138,NaN,mathqa_train_37
7,83a080372c1d,"LINEAR:add(ARG0,CONST_1)",116,NaN,116,NaN,mathqa_train_552
8,024224bbcc9b,"LINEAR:multiply(ARG1,CONST_0_2778)|divide(ARG0...",110,NaN,110,NaN,mathqa_train_261
9,59ad42bbd54f,"LINEAR:multiply(ARG0,ARG1)|add(ARG2,TEMP0)",106,NaN,106,NaN,mathqa_train_10


In [49]:
# Cell 9: limitations & next automated improvements (run or do manually)
notes = {
    "limitations": [
        "annotated_formula parser is a simple function-call parser and will fail on malformed inputs.",
        "ARG -> numeric mapping is heuristic: assigns ARG0 -> first numeric token in problem text. This is often correct but not always.",
        "infix conversion covers common binary ops (add/sub/mult/div/power). Other domain-specific ops remain as func(...) strings.",
        "we do not yet reliably infer 'solve_for' variable or a formal problem 'type' (quadratic, linear eqn, geometry, etc.)."
    ],
    "next_steps": [
        "Review top templates and correct/unify templates that differ only by constant token styles.",
        "Improve ARG mapping using 'linear_formula' when available (it gives ordering of arguments) — we can parse linear_formula to get explicit ARG->n mapping.",
        "Add rules to infer problem type by pattern-matching ops (e.g., contains 'power' and 'add' -> polynomial; contains 'circle_area' -> geometry/circle).",
        "Create a small human-validated mapping for top ~200 templates to assign semantic role names for slots (e.g., ARG0 -> QUANTITY_A, ARG1 -> RATE).",
        "Add extraction of numeric units (e.g., 'm', 'kg') using simple heuristics from the problem text and attach to arg slots."
    ]
}
print(json.dumps(notes, indent=2))


{
  "limitations": [
    "annotated_formula parser is a simple function-call parser and will fail on malformed inputs.",
    "ARG -> numeric mapping is heuristic: assigns ARG0 -> first numeric token in problem text. This is often correct but not always.",
    "infix conversion covers common binary ops (add/sub/mult/div/power). Other domain-specific ops remain as func(...) strings.",
    "we do not yet reliably infer 'solve_for' variable or a formal problem 'type' (quadratic, linear eqn, geometry, etc.)."
  ],
  "next_steps": [
    "Review top templates and correct/unify templates that differ only by constant token styles.",
    "Improve ARG mapping using 'linear_formula' when available (it gives ordering of arguments) \u2014 we can parse linear_formula to get explicit ARG->n mapping.",
    "Add rules to infer problem type by pattern-matching ops (e.g., contains 'power' and 'add' -> polynomial; contains 'circle_area' -> geometry/circle).",
    "Create a small human-validated mapping for